In [55]:
import pandas
from bs4 import BeautifulSoup
import  os
import json
import re
import urllib as urllib2
from urllib import request
import requests
from time import sleep
from selenium import webdriver


In [2]:
BASE_URL = 'http://www.reformagkh.ru'
AREAS_URL = 'https://www.reformagkh.ru/myhouse?tid=2280999&sort=alphabet&item=mkd&ajax'
AREAS_FILE = 'data/raw/regions.json'
MUN_URLPAT = 'http://www.reformagkh.ru/myhouse?tid=2280999&sort=alphabet&item=mkd&ajax&tid=%s'
MUN_FILEPAT = 'data/raw/tid_%s.json'
BUILDING_LIST_PAT = 'http://www.reformagkh.ru/myhouse/list?tid=%s&sort=alphabet&item=tp&mkdsort=name&mkdorder=asc&perpage=10000'
BUILDING_URLPAT = 'http://www.reformagkh.ru/myhouse/view/%s/?group=0'
BUILDING_KEYS = ['house_id', 'url', 'address','year', 'area', 'num_people', 'status']

In [3]:
def table2dict(tab, skip=0, keys=[], keyshift=2):
    rows = []
    i = 0
    for k in tab.findAll('tr', recursive=False):
        i += 1
        if i - skip <= 0: continue
        row = {}
        #print k
        tds = k.findAll('td')
        url=tds[0].find('a')['href']
        row['url']=BASE_URL+url
        row['house_id']=re.search('/\d+/',url).group(0).replace('/','')
        tdslen = len(tds)
        for n in range(0, len(keys), 1):
            try:
                row[keys[n+keyshift]] = tds[n].text
            except:
                pass
        #print row
        rows.append(row)
    return rows

In [53]:
def extract_areas():
    """Extracts indicators data by area code. Saves as JSON file"""
    filename = AREAS_FILE
    if os.path.exists(filename): return None
    print("Processing Areas", AREAS_URL)
    
    url = AREAS_URL
    browser = webdriver.PhantomJS(executable_path="C:\\Users\\772\\Documents\\phantomjs-2.1.1-windows\\bin\\phantomjs.exe")
    browser.get(url)
    sleep(2)
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    body = soup.find('div', attrs={'class':'col-4 py-4'})
    print(body)
    #body = body.find_all('a')

    #response = requests.get(AREAS_URL)
    #page = BeautifulSoup(response.content)
    #content = soup.find_all('div')
    #content = soup.find('div', attrs={'class' : 'tables'}, recursive=True).find_all('a')
    areas=[]
    for a in content:
        tid=a['href'].split('=')[1]
        name=str(a.contents[0])
        areas.append({'name':name,'tid':tid})
        print(tid,name)       
    f = open(filename, 'w')
    f.write(json.dumps(areas, indent=4))
    f.close()
    print("Processing Areas", 'written')


In [54]:
extract_areas()

Processing Areas https://www.reformagkh.ru/myhouse?tid=2280999&sort=alphabet&item=mkd&ajax
<div class="col-4 py-4">
<div class="d-flex align-items-top">
<span class="col-2 my-3 f-16 text-black-grey">В</span>
<div class="w-100">
<div class="d-flex justify-content-between align-items-top my-3">
<span class="d-flex align-items-top" title="Средний процент раскрытия информации по МКД субъекта РФ ниже 60%">
<svg class="mr-2 flex-shrink-0 my-2" fill="none" height="10" title="Средний процент раскрытия информации по МКД субъекта РФ ниже 60%" viewbox="0 0 8 8" width="10" xmlns="http://www.w3.org/2000/svg">
<path clip-rule="evenodd" d="M4 8C6.20914 8 8 6.20914 8 4C8 1.79086 6.20914 0 4 0C1.79086 0 0 1.79086 0 4C0 6.20914 1.79086 8 4 8Z" fill="" fill-rule="evenodd"></path>
</svg>
<span>Внуковское</span>
</span>
<span class="ml-5 text-grey text-nowrap">0</span>
</div>
<div class="branch pl-5"></div>
<div class="d-flex justify-content-between align-items-top my-3">
<span class="d-flex align-items-to

NameError: name 'content' is not defined

In [79]:
def extract_municipal():
    """Extracts data about municipal areas"""
    f = open(AREAS_FILE, 'r')
    data = f.read()
    f.close()
    js = json.loads(data)
    for record in js:
        filename = MUN_FILEPAT % record['tid']
        if os.path.exists(filename):
            print "Processing. TID %s" % record['tid'], 'skipped'
            continue
        print MUN_URLPAT % record['tid']
        u = urllib2.urlopen(MUN_URLPAT % record['tid'])
        data = u.read()
        u.close()
        soup = BeautifulSoup(data)
        div_muni=soup.find('div', attrs={'class' : 'tables'}, recursive=True)
        div_buildings=soup.find('div', attrs={'id' : 'printAnketa'}, recursive=True)
        mun_areas=[]
        if div_muni:
            content = div_muni.findAll('a')
            for a in content:
                tid=a['href'].split('=')[1]
                name=str(a.contents[0])
                mun_areas.append({'name':name,'tid':tid})
                print tid,name        
        else:
            if div_buildings:
                mun_areas.append({'name':record['name'],'tid':record['tid']})
                print record['tid'],record['name']
            else:
                print "Something wrong :-("
                break
        f = open(MUN_FILEPAT % record['tid'], 'w')
        f.write(json.dumps(mun_areas, indent=4))
        f.close()
        print "Processing. TID %s" % record['tid'], 'written'



SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Processing. TID %s" % record['tid'], 'skipped')? (<ipython-input-79-f5979c800570>, line 10)

In [58]:
def get_buildings_list(url, getlast=True):
    url = AREAS_URL
    browser = webdriver.PhantomJS(executable_path="C:\\Users\\772\\Documents\\phantomjs-2.1.1-windows\\bin\\phantomjs.exe")
    browser.get(url)
    sleep(2)
    html = browser.page_source
    
    soup = BeautifulSoup(html)
    content = soup.find('div', attrs={'id' : 'printAnketa'}, recursive=True)
#    print url, content
#    print soup
    if getlast:
        itemlist = content.find('div', attrs={'class' : 'item-list'})
        pager = itemlist.find('ul', attrs={'class' : 'pager'})
        plast = pager.find('li', attrs={'class' : 'pager-last'})
        if plast is not None:
            lastpagenum = plast.find('a').string
        else:
            pitems = pager.findAll('li', attrs={'class' : 'pager-item'})
            lastpagenum = pitems[-1].text
    else:
        lastpagenum = None

    tab = content.find('table')
    if tab is not None:
        rows = table2dict(tab, skip=1, keys=BUILDING_KEYS)
    else:
        rows = []
    return rows, lastpagenum

In [61]:
def save_buildings(tid =None):
    """Saves buildings data by TID"""
    filename = 'data/raw/buildings_%s.json' % 'lomonosovsky'
    #if os.path.exists(filename):
    #    print "Skipping buildings for mun %s" % tid
    #    return
    #print "Processing buildings for mun %s" % tid
    url = 'https://www.reformagkh.ru/myhouse?tid=2281077&item=tp'
    #url = BUILDING_LIST_PAT % tid
    rows, lastp =  get_buildings_list(url, getlast=False)
    #print '- processed', url
    f = open(filename, 'w')
    json.dump(rows, f, indent=4)
    f.close()

In [62]:
save_buildings()

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
def extract_buildings_list():
    """Extracts list of buildings for each area"""
    f = open(AREAS_FILE, 'r')
    data = f.read()
    f.close()
    areas = json.loads(data)
    for area in areas:
        f = open(MUN_FILEPAT % area['tid'], 'r')
        data = f.read()
        f.close()
        try:
            muns = json.loads(data)
        except:
            continue
        for mun in muns:
            save_buildings(mun['tid'])
        save_buildings(area['tid'])


In [ ]:

def get_buildings_data(tid):
    """Collect buildings data for specific area by TID (territory id)"""
    filename = 'data/raw/buildings_%s.json' % tid
    f = open(filename, 'r')
    rows = json.load(f)
    f.close()
    return rows

In [ ]:

def refine_buildings():
    """Prepare data for publishing"""
    idkeys = []
    allrows = []
    allterr = []
    f = open(AREAS_FILE, 'r')
    data = f.read()
    f.close()
    areas = json.loads(data)
    for area in areas:
        f = open(MUN_FILEPAT % area['tid'], 'r')
        data = f.read()
        f.close()
        try:
            muns = json.loads(data)
        except:
            continue
        for mun in muns:
            allterr.append({'name' : mun['name'], 'id' : mun['tid']})
            rows = get_buildings_data(mun['tid'])
            for row in rows:
                if row['house_id'] not in idkeys: idkeys.append(row['house_id'])
                else: continue
                item = {}
                item['area_name'] = mun['name']
                item['area_id'] = mun['tid']
                item['address'] = row['address']
                item['house_year'] = row['year']
                item['house_id'] = row['house_id']
                item['house_area'] = row['area']
                item['num_people'] = row['num_people']
                allrows.append(item)
        rows = get_buildings_data(area['tid'])
        if len(rows) > 0:
            allterr.append({'name' : area['name'], 'id' : area['tid']})
        for row in rows:
            if row['house_id'] not in idkeys: idkeys.append(row['house_id'])
            else: continue
            item = {}
            item['area_name'] = mun['name']
            item['area_id'] = mun['tid']
            item['address'] = row['address']
            item['house_year'] = row['year']
            item['house_id'] = row['house_id']
            item['house_area'] = row['area']
            item['num_people'] = row['num_people']
            allrows.append(item)
    f = open('data/refine/buildings.json', 'w')
    json.dump(allrows, f, indent=4)
    f.close()
    f = open('data/refine/areas.json', 'w')
    json.dump(allterr, f, indent=4)
    f.close()


Processing Areas http://www.reformagkh.ru/myhouse?tid=2280999&sort=alphabet&item=mkd&ajax
<html><head><script lang="text/javascript">
    var checkKey = 'a0ee50584a39e6d7c4e6e3191b2cf8dd',
        key1 = '7b2808d07931224c27054f522a3395d15b3100ea640576bcc08f',
        val1 = Number('1060883614'),
        key2 = '7b2808d07931224c27054f522a3395d13ee9da1dd87b63c6f83c',
        val2 = Number('1351839011');

    var xhrRequest = new XMLHttpRequest();

    xhrRequest.open('POST', '/save-check-data');
    xhrRequest.setRequestHeader('Content-Type', 'application/x-www-form-urlencoded');
    xhrRequest.setRequestHeader('X-Requested-With', 'XMLHttpRequest');
    xhrRequest.onload = function () {
        var response = JSON.parse(xhrRequest.responseText || '{}');

        if (xhrRequest.status === 200 && !!response.success) {
            window.location.reload();
        } else {
            window.location.href = '/';
        }
    };
    xhrRequest.send(encodeURI('key1=' + key1 + '&key2=' + key2

AttributeError: 'NoneType' object has no attribute 'find_all'